In [7]:
# Uncomment to install dependencies
#!pip install pandas
#!pip install youtube_dl
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [8]:
import pandas as pd
import random
import json
import os
from youtube_dl import YoutubeDL

from google_auth_oauthlib.flow import InstalledAppFlow
from apiclient.discovery import build
from apiclient.http import MediaFileUpload
from apiclient import errors

yt_url_prefix = 'https://www.youtube.com/watch?v='
output_folder = 'outputs/'

In [17]:
# GOOGLE DRIVE API CONSTANTS
flow = InstalledAppFlow.from_client_secrets_file('credentials.json', ['https://www.googleapis.com/auth/drive'])
creds = flow.run_local_server(port=0)
folder_id = '1UnDIe4VHMM8bZzIfKEG8NJdQGnNrrxEG'
drive_api = build('drive', 'v3', credentials=creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=786879558582-eh0bho168rp2k1qord3av8r5u10q64sl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A56858%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=Lq5R98UsAek63CkDGzxWZ53RiZXgkC&access_type=offline


In [34]:
# Helpful function
def get_file_list_from_folder(service, folder_id):
    """Print files belonging to a folder.

    Args:
    service: Drive API service instance.
    folder_id: ID of the folder to print files from.
    """
    kwargs = {
        "q": "'{}' in parents".format(folder_id)
    }
    request = service.files().list(**kwargs)
    files = []
    while request is not None:
        response = request.execute()
        # Do stuff with response['files']
        files.extend(response['files'])
        request = service.files().list_next(request, response)
    return files

In [35]:
trailers = pd.read_csv("trailers.csv", index_col=0)
#trailers

In [40]:
ydl_opts = {
    'outtmpl': output_folder + '%(id)s.%(ext)s',
    'format': 'bestvideo'
}

uploaded = get_file_list_from_folder(drive_api, folder_id)
#print(uploaded)
uploaded = [d['name'].split('.')[0] for d in uploaded]
duplicates = pd.Series(uploaded).value_counts()
uploaded = set(uploaded)
#print(duplicates)
#print(f'files already uploaded {uploaded}')

all_to_download = set(trailers['youtube_key'])
#print(f'files already uploaded & requested for upload{uploaded & all_to_download}')
remaining_trailers = all_to_download - uploaded
print(f'remaining trailers (len({len(remaining_trailers)})) {remaining_trailers}')

remaining trailers (len(1507)) {'mp2SbaK8dDg', '8VOAxzgq42A', 'tFMo3UJ4B4g', 'uW_GNKVuZ1c', 'VdCRypsfZDE', 'IRVJwH3hBd0', 'ajhZRL1j7a0', 'MJsDuGRiLqM', 'ePbKGoIGAXY', 'F5-Lv4CJmFM', 'tx6_JMDFfWI', 'Ymoh5SIqgtw', 'rl2bysiyltg', 'foiRWtJTmo0', 'V75dMMIW2B4', 'fcb68kAOvt4', 'rk-dF1lIbIg', 'uSDNZeRX_1Y', 'Fxp32VHaYdE', 'eOrNdBpGMv8', 'dtKMEAXyPkg', '6acRHWnfZAE', 'gJAMYJkA2Uc', 'vco0SpSz17g', 'HdVvjvW_OEo', 'aFtzNHTB4C0', 'K4xD8ZMdJms', 'SbXIj2T-_uk', 'NQu-153MnGQ', '-qJjiq72WOo', 'qP755JkDxyM', 'IIEmUFGJPSw', 'CZIt20emgLY', 'tW17tB8Xwkw', 'xBOAmtjsZeA', 'fAIX12F6958', 'gKTVQPOH8ZA', '5bP1f_1o-zo', 'uEXsOqdzYhE', 'PYPK3jPZkYI', '6SKI9rgqFck', 'oJsRRVXNj4M', '34cEo0VhfGE', 'wmiIUN-7qhE', 'uJQZdk5jMEg', 'AGACeWVdFqo', 'rTL1FmvVCuA', 'gsYKF8ecC8g', 'b_KllgySsPo', 'n6BVyk7hty8', 'jOGDozdW9Yo', 'OvgyXKDXdZY', 'S3AVcCggRnU', '0XAlutSwY0I', '4bqeLmhNbfI', 'hWjlUj7Czlk', 'jN4mg2rESLU', 'qLTDtbYmdWM', 'ue80QwXMRHg', 'rNhV8dk6W2M', 'SCTzYY95Aw4', '2-2x3r1m2I4', 'vmSpCLefjnw', 'ekshABiptOY', 'nijccxW

In [ ]:
with YoutubeDL(ydl_opts) as ydl:
    for t in remaining_trailers:
        try:
            # Helpful Constants
            yt_link = yt_url_prefix + t

            # Download Video
            print(f"Downloading {t}")
            trailer_yt_info = ydl.extract_info(yt_link, download=True)
            
            # Determine File Name
            file_name = [f for f in os.listdir(output_folder) if t in f][0]
            print(file_name)

            # Upload to Drive
            print(f"Uploading file {file_name}...")
            body = {'name': file_name, 'parents': [folder_id]}
            media = MediaFileUpload(output_folder + file_name)
            fiahl = drive_api.files().create(body=body, media_body=media).execute()
            print(f"Created file '{fiahl.get('name')}' id '{fiahl.get('id')}'.")

            # Delete video from hard drive
            os.remove(output_folder + file_name)
            print(f"Removed {file_name}")
            print("Success!")
        except Exception as e:
            print(f"Failed with exception {e}")
        print()
        


[youtube] mp2SbaK8dDg: Downloading webpage
[youtube] mp2SbaK8dDg: Downloading video info webpage
[download] outputs/mp2SbaK8dDg.mp4 has already been downloaded
[download] 100% of 56.20MiB
mp2SbaK8dDg.mp4
Uploading file mp2SbaK8dDg.mp4...
Created file 'mp2SbaK8dDg.mp4' id '1bEv5uG8j0eqv0OXZ5e3Vg9Ebz9hW2DJQ'.
Removed mp2SbaK8dDg.mp4
Success!

[youtube] 8VOAxzgq42A: Downloading webpage
[youtube] 8VOAxzgq42A: Downloading video info webpage
[download] outputs/8VOAxzgq42A.mp4 has already been downloaded
[download] 100% of 35.17MiB
8VOAxzgq42A.mp4
Uploading file 8VOAxzgq42A.mp4...
Created file '8VOAxzgq42A.mp4' id '1i5qGnH8pYxR1rwlCuAJgH0BEJzjaIgE4'.
Removed 8VOAxzgq42A.mp4
Success!

[youtube] tFMo3UJ4B4g: Downloading webpage
[youtube] tFMo3UJ4B4g: Downloading video info webpage
[download] outputs/tFMo3UJ4B4g.webm has already been downloaded
[download] 100% of 13.86MiB
tFMo3UJ4B4g.webm
Uploading file tFMo3UJ4B4g.webm...
Created file 'tFMo3UJ4B4g.webm' id '1Jo0Gj68hWz0WWBX5_q_nrofEVLQsze32'.
R